In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","6g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 16:56:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [3]:
sc = spark.sparkContext

In [4]:
# pobranie spakowanego zbioru za pomocą polecenia systemowego wget
# strona datasetu: https://archive.ics.uci.edu/dataset/911/recipe+reviews+and+user+feedback+dataset
!wget https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

--2024-12-16 16:56:49--  https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip
128.195.10.252ive.ics.uci.edu (archive.ics.uci.edu)... 
connected. to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... 
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘recipe+reviews+and+user+feedback+dataset.zip’

recipe+reviews+and+     [     <=>            ]   2.02M  1.52MB/s    in 1.3s    

2024-12-16 16:56:51 (1.52 MB/s) - ‘recipe+reviews+and+user+feedback+dataset.zip’ saved [2114088]



In [5]:
# listujemy zawartość bieżącego folderu
!ls

data	      Lab6.ipynb				    Zad1.ipynb
employee.csv  recipe+reviews+and+user+feedback+dataset.zip  Zad2.ipynb
lab	      recipe_reviews.zip


In [6]:
# zmiana nazwy pliku - nie jest konieczna, ale trzeba zmienić później ścieżkę w kolejnej komórce notatnika
!mv recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

In [7]:
# wypakowujemy plik do podfolderu data
import zipfile
with zipfile.ZipFile("recipe_reviews.zip", 'r') as zip_ref:
    zip_ref.extractall("./lab")

In [8]:
!ls ./data

 parquet  'Recipe Reviews and User Feedback Dataset.csv'


In [9]:
# sprawdzamy jak wyglądają 3 pierwsze linie pliku, widać, że pierwsza zawiera nagłówki kolumn a dane są oddzielone przecinkiem
!head -3 "lab/Recipe Reviews and User Feedback Dataset.csv"

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because of onion haters in my house, used Italian seasoning instead of just oregano, and use a paprika/ cayenne mix and a little more than the recipe called for.. we like everything a bit more hot. The chili was amazing! It was easy to make and everyone absolutely loved it. It will now be a staple meal in our house."
1,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,"Bush used to have a white chili bean and it made this recipe super simple. I’ve written to them and asked them to please!, bring them back"


In [10]:
df_reviews = spark.read.csv('./lab/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")

In [11]:
# Zad 1
# a
df_reviews = df_reviews.withColumnRenamed('_c0', 'id')

In [12]:
# b
df_reviews.select('reply_count').orderBy(df_reviews['reply_count'], ascending=False).show(10)

+-----------+
|reply_count|
+-----------+
|          3|
|          3|
|          3|
|          3|
|          3|
|          3|
|          2|
|          2|
|          2|
|          2|
+-----------+
only showing top 10 rows



In [13]:
# c
from pyspark.sql.functions import col

# Rzutowanie kolumny 'best_score' na typ numeryczny (np. IntegerType)
df_reviews = df_reviews.withColumn('best_score', col('best_score').cast('int'))

# Teraz wykonaj grupowanie i sumowanie
df_reviews.groupBy('recipe_code').sum('best_score').orderBy('sum(best_score)', ascending=False).show(10)

+-----------+---------------+
|recipe_code|sum(best_score)|
+-----------+---------------+
|       2832|          98863|
|      14299|          85497|
|      17826|          64880|
|       3309|          64247|
|      21444|          60755|
|      32480|          59867|
|      12540|          59195|
|       2912|          54032|
|      42083|          51975|
|      19731|          47905|
+-----------+---------------+
only showing top 10 rows



In [14]:
# d
# Rzutowanie kolumny 'reply_count' na typ numeryczny (np. IntegerType)
df_reviews = df_reviews.withColumn('reply_count', col('reply_count').cast('int'))

df_reviews.groupBy('recipe_code').sum('reply_count').orderBy('sum(reply_count)', ascending=False).show(10)

+-----------+----------------+
|recipe_code|sum(reply_count)|
+-----------+----------------+
|       2832|              16|
|        414|              13|
|      32480|              12|
|      14299|              10|
|      18345|              10|
|       8202|               9|
|      12003|               8|
|      41095|               8|
|       1324|               8|
|       8431|               6|
+-----------+----------------+
only showing top 10 rows



In [15]:
# e
df_reviews.groupBy('stars').count().show()

+-----+-----+
|stars|count|
+-----+-----+
|    3|  490|
|    0| 1696|
| NULL|   86|
|    5|13829|
|    1|  280|
|    4| 1655|
|    2|  232|
+-----+-----+



In [16]:
sc.stop()